<a href="https://colab.research.google.com/github/FatimaJHussain/Amazon_Vine_Analysis/blob/main/Amazon_Reviews_ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.1.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-04-18 01:05:37--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.77MB/s    in 0.2s    

2021-04-18 01:05:38 (5.77 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://uftchallenge16.s3.us-east-2.amazonaws.com/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Wireless_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

### Create DataFrames to match tables

In [9]:
from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame.    
from pyspark.sql.types import IntegerType

df=df.withColumn("customer_id", df["customer_id"].cast(IntegerType()))
df=df.withColumn("product_parent", df["product_parent"].cast(IntegerType()))
df=df.withColumn("star_rating", df["star_rating"].cast(IntegerType()))
df=df.withColumn("helpful_votes", df["helpful_votes"].cast(IntegerType()))
df=df.withColumn("total_votes", df["total_votes"].cast(IntegerType()))
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   16414143|R3W4P9UBGNGH1U|B00YL0EKWE|     852431543|LG G4 Case Hard T...|        Wireless|          2|            1|          3|   N|                Y|Looks good, funct...|2 issues  -  Once...| 2015-08-31|
|         US|   50800750|R15V54KBMTQWAY|B00XK95RPQ|     516894650|Selfie Stick Fibl...|        Wireless|          4|    

In [30]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [17]:
# Create the customers_table DataFrame

#customers_table_df=df.select(['customer_id', 'star_rating']).describe().show()
#customers_df = df.groupby("customer_id").agg({""}).withColumnRenamed("customer_id", "customer_count")
customers_table = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_table.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   46909180|             6|
|   42560427|             7|
|   43789873|             3|
|   22037526|             2|
|   34220092|             2|
|   42801586|             1|
|    9565734|             2|
|   15829398|             1|
|   38247118|             1|
|   32478248|             2|
|   48114630|             1|
|   23085063|             1|
|   32787070|             3|
|   43515569|             1|
|    4919528|             2|
|    5088547|             2|
|   41852407|             3|
|   49703087|             1|
|   12713799|             1|
|   36728141|             8|
+-----------+--------------+
only showing top 20 rows



In [33]:
# Create the products_table DataFrame and drop duplicates. 
 
#products_table_df=df.select(['product_id', 'product_title']).describe().show()
products_table = df.select(['product_id', 'product_title']).dropDuplicates(['product_id'])
products_table.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1060104342|Premium Asus MeMO...|
|159775045X|National Geograph...|
|5891090414|           STAND-N02|
|6175001532|Colorful Abstract...|
|9572548964|Pink Flower Purpl...|
|9713000463|Rapid Charger KIT...|
|9838427853|PowerBear® Samsun...|
|9861033181|Purple Flower Blu...|
|9983781239|PREMIUM USB Adapt...|
|9984225437|Sanyo MM-8300 Sta...|
|B00000J0IV|Magellan Leather ...|
|B00000J1PX|Cobra MAVOX Voice...|
|B00003OPEV|Garmin eMap Delux...|
|B0000520T4|Ericsson 650 mAh ...|
|B00005I9QK|Aquapac Waterproo...|
|B00005QT35|Garmin GPS 45 Per...|
|B00006JP9X|AMERICAN INTERNAT...|
|B00007EDL7|Sony DCC-E34CP Po...|
|B00008E1OH|Samsung Leather C...|
|B00008RCQS|Virgin Mobile Pay...|
+----------+--------------------+
only showing top 20 rows



In [20]:

from pyspark.sql.functions import countDistinct
products_table.select(countDistinct('product_id')).show()

+--------------------------+
|count(DISTINCT product_id)|
+--------------------------+
|                    906592|
+--------------------------+



In [21]:
products_table.count()

906687

In [9]:
# Create the review_id_table DataFrame. 
from pyspark.sql.functions import to_date

#review_id_table=df.select(['review_id','product_id', 'customer_id','product_parent', 'review_date']).describe().show()
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")

# review_id_df = df.select([, to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table=df.select(['review_id','product_id', 'customer_id','product_parent', to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_table.show()

+--------------+----------+-----------+--------------+-----------+
|     review_id|product_id|customer_id|product_parent|review_date|
+--------------+----------+-----------+--------------+-----------+
|R3W4P9UBGNGH1U|B00YL0EKWE|   16414143|     852431543| 2015-08-31|
|R15V54KBMTQWAY|B00XK95RPQ|   50800750|     516894650| 2015-08-31|
| RY8I449HNXSVF|B00SXRXUKO|   15184378|     984297154| 2015-08-31|
|R18TLJYCKJFLSR|B009V5X1CE|   10203548|     279912704| 2015-08-31|
|R1NK26SWS53B8Q|B00D93OVF0|     488280|     662791300| 2015-08-31|
|R11LOHEDYJALTN|B00XVGJMDQ|   13334021|     421688488| 2015-08-31|
|R3ALQVQB2P9LA7|B00KQW1X1C|   27520697|     554285554| 2015-08-31|
|R3MWLXLNO21PDQ|B00IP1MQNK|   48086021|     488006702| 2015-08-31|
|R2L15IS24CX0LI|B00HVORET8|   12738196|     389677711| 2015-08-31|
|R1DJ8976WPWVZU|B00HX3G6J6|   15867807|     299654876| 2015-08-31|
|R3MRWNNR8CBTB7|B00U4NATNQ|    1972249|     577878727| 2015-08-31|
|R1DS6DKTUXAQK3|B00SZEFDH8|   10956619|     654620704| 2015-08

In [10]:
# Create the vine_table. DataFrame

vine_table=df.select(['review_id','verified_purchase', 'total_votes','helpful_votes', 'vine','star_rating'])
vine_table.show()
# vine_df = df.select([])


+--------------+-----------------+-----------+-------------+----+-----------+
|     review_id|verified_purchase|total_votes|helpful_votes|vine|star_rating|
+--------------+-----------------+-----------+-------------+----+-----------+
|R3W4P9UBGNGH1U|                Y|          3|            1|   N|          2|
|R15V54KBMTQWAY|                N|          0|            0|   N|          4|
| RY8I449HNXSVF|                Y|          0|            0|   N|          5|
|R18TLJYCKJFLSR|                Y|          0|            0|   N|          5|
|R1NK26SWS53B8Q|                Y|          0|            0|   N|          5|
|R11LOHEDYJALTN|                Y|          0|            0|   N|          5|
|R3ALQVQB2P9LA7|                Y|          0|            0|   N|          4|
|R3MWLXLNO21PDQ|                Y|          0|            0|   N|          5|
|R2L15IS24CX0LI|                Y|          0|            0|   N|          5|
|R1DJ8976WPWVZU|                Y|          0|            0|   N

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.cfkodobwrlh6.us-east-2.rds.amazonaws.com:5432/challenge16"
config = {"user":"postgres", 
          "password": "bootcamp", 
          "driver":"org.postgresql.Driver"}

In [13]:
# Write review_id_df to table in RDS
review_id_table.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [34]:
%%time
# Write products_df to table in RDS
# about 3 min
products_table.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

CPU times: user 1.53 s, sys: 225 ms, total: 1.75 s
Wall time: 7min 7s


In [18]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_table.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [32]:
# Write vine_df to table in RDS
# 50 minutes
vine_table.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)